In [ ]:
# app.py

import plotly.graph_objects as go
import plotly.figure_factory as ff
from dash import Dash, dcc, html, Output, Input, State
from simulation import SupplyChainSimulation
import networkx as nx  # For network visualizations
import json

# Initialize the Dash app
app = Dash(__name__)
server = app.server  # For deploying to platforms like Heroku

# Initialize the layout with simulation controls and visualizations
app.layout = html.Div([
    html.H1("Supply Chain Simulation Dashboard"),
    
    # Simulation Controls
    html.Div([
        html.Div([
            html.Label("Number of Products:"),
            dcc.Input(id='num-products', type='number', value=10, min=1, max=100),
        ], style={'marginRight': '20px'}),
        
        html.Div([
            html.Label("Success Rate (%):"),
            dcc.Slider(
                id='success-rate',
                min=0,
                max=100,
                step=1,
                value=90,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                tooltip={"placement": "bottom", "always_visible": True},
            ),
        ], style={'width': '60%'}),
        
        html.Button("Run Simulation", id='run-simulation', n_clicks=0, style={'marginTop': '20px'}),
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'space-between', 'marginBottom': '40px'}),
    
    # Simulation Status
    html.Div(id='simulation-status', style={'whiteSpace': 'pre-line', 'marginBottom': '20px'}),
    
    # Visualizations
    dcc.Tabs(id='tabs', value='tab-1', children=[
        dcc.Tab(label='Product Flow', value='tab-1'),
        dcc.Tab(label='Blockchain Structure', value='tab-2'),
        dcc.Tab(label='Network of Entities', value='tab-3'),
        dcc.Tab(label='Product Lifecycle Gantt Chart', value='tab-4'),
    ]),
    html.Div(id='tabs-content'),
    
    # Store for simulation data
    dcc.Store(id='simulation-data', storage_type='memory'),
])

# Serialization Functions
def serialize_blockchain(blockchain):
    """Serialize the blockchain data into a list of dictionaries."""
    chain_data = []
    for block in blockchain:
        chain_data.append({
            'index': block['index'],
            'previous_hash': block['previous_hash'],
            'timestamp': block['timestamp'],
            'transactions': block['transactions'],
            'nonce': block['nonce'],
            'hash': block['hash']
        })
    return chain_data

def serialize_products(products):
    """Serialize the products data into a dictionary."""
    products_data = {}
    for product_id, product in products.items():
        products_data[product_id] = product  # Assuming product is already a dict
    return products_data

def visualize_blockchain_data(blockchain_data, orientation='horizontal'):
    """Create a Plotly graph for the blockchain structure with a linear layout."""
    G = nx.DiGraph()
    labels = {}
    
    for block in blockchain_data:
        index = block['index']
        G.add_node(index)
        labels[index] = f"Block {index}\nHash: {block['hash'][:6]}..."
        if index > 0:
            G.add_edge(index - 1, index)
    
    # Custom Linear Layout
    pos = {}
    spacing = 1.0  # Adjust spacing between nodes
    for node in G.nodes():
        if orientation == 'horizontal':
            pos[node] = (node * spacing, 0)  # Horizontal layout
        elif orientation == 'vertical':
            pos[node] = (0, node * spacing)  # Vertical layout
        else:
            raise ValueError("Orientation must be 'horizontal' or 'vertical'")
    
    # Edges
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=2, color='gray'),
        hoverinfo='none',
        mode='lines')
    
    # Nodes
    node_x = []
    node_y = []
    node_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(labels[node])
    
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="bottom center" if orientation == 'horizontal' else "middle right",
        hoverinfo='text',
        marker=dict(
            color='skyblue',
            size=20,
            line_width=2))
    
    # Figure Layout Adjustments
    if orientation == 'horizontal':
        x_range = [min(node_x) - spacing, max(node_x) + spacing]
        y_range = [min(node_y) - spacing, max(node_y) + spacing]
    else:
        x_range = [min(node_x) - spacing, max(node_x) + spacing]
        y_range = [min(node_y) - spacing, max(node_y) + spacing]
    
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Blockchain Structure (Linear Layout)',
                        showlegend=False,
                        hovermode='closest',
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=x_range),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, range=y_range),
                        margin=dict(l=40, r=40, t=40, b=40))
                   )
    return fig

def visualize_entity_network_data(products_data):
    """Create a Plotly graph for the network of entities."""
    G = nx.MultiDiGraph()
    for product_id, product in products_data.items():
        history = product['history']
        for i in range(len(history) - 1):
            source = history[i]['updated_by']
            target = history[i+1]['updated_by']
            G.add_edge(source, target, product=product_id)

    pos = nx.spring_layout(G, k=0.5, iterations=50)

    # Edges
    edge_traces = []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_traces.append(go.Scatter(
            x=[x0, x1, None], y=[y0, y1, None],
            line=dict(width=1, color='gray'),
            hoverinfo='text',
            mode='lines',
            text=f"Product: {edge[2]['product']}"))

    # Nodes
    node_x = []
    node_y = []
    node_text = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(node)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="top center",
        hoverinfo='text',
        marker=dict(
            color='lightgreen',
            size=10,
            line_width=2))

    # Figure
    fig = go.Figure(data=edge_traces + [node_trace],
                    layout=go.Layout(
                        title='Network of Entities',
                        showlegend=False,
                        hovermode='closest'))
    return fig

def visualize_product_flow_data(products_data):
    """Create a Plotly Sankey diagram for product flow."""
    sources = []
    targets = []
    values = []
    label_list = []

    for product_id, product in products_data.items():
        history = product['history']
        for i in range(len(history) - 1):
            source = history[i]['updated_by']
            target = history[i+1]['updated_by']
            if source not in label_list:
                label_list.append(source)
            if target not in label_list:
                label_list.append(target)
            sources.append(label_list.index(source))
            targets.append(label_list.index(target))
            values.append(1)

    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            label=label_list,
            color="blue"),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color="lightblue"))])

    fig.update_layout(title_text="Product Flow Through Entities", font_size=10)
    return fig

def generate_gantt_chart_data(products_data):
    """Create a Plotly Gantt chart for product lifecycle."""
    df = []
    for product_id, product in products_data.items():
        history = product['history']
        for i, event in enumerate(history):
            # Parse the event date
            start_time = event['date']  # Assuming date is in ISO format
            if i + 1 < len(history):
                finish_time = history[i + 1]['date']
            else:
                # If it's the last event, add a small delta to the finish time
                from datetime import datetime, timedelta
                finish_time_dt = datetime.strptime(start_time, '%Y-%m-%dT%H:%M:%S') + timedelta(hours=1)
                finish_time = finish_time_dt.strftime('%Y-%m-%dT%H:%M:%S')
            df.append({
                'Task': product_id,
                'Start': start_time,
                'Finish': finish_time,
                'Resource': event['status']
            })

    # Assign colors to resources
    colors = {
        'Created': 'rgb(46, 137, 205)',
        'Manufactured': 'rgb(114, 44, 121)',
        'In Transit': 'rgb(198, 47, 105)',
        'Available for Sale': 'rgb(58, 149, 136)',
        'Purchased': 'rgb(107, 127, 135)'
    }

    fig = ff.create_gantt(
        df,
        index_col='Resource',
        show_colorbar=True,
        group_tasks=True,
        colors=colors,
        title='Product Lifecycle Gantt Chart'
    )
    fig.update_layout(
        xaxis_title='Time',
        yaxis_title='Products',
        height=600,
        margin=dict(l=50, r=50, t=50, b=50)
    )
    return fig

# Callback to run simulation and store data
@app.callback(
    Output('simulation-data', 'data'),
    Output('simulation-status', 'children'),
    Input('run-simulation', 'n_clicks'),
    State('num-products', 'value'),
    State('success-rate', 'value'),
    prevent_initial_call=True
)
def run_simulation_callback(n_clicks, num_products, success_rate):
    if n_clicks < 1:
        # No simulation run yet
        return {}, "Click 'Run Simulation' to start."

    # Run the simulation
    simulation = SupplyChainSimulation(success_rate=success_rate / 100)  # Convert to probability
    scm = simulation.run(num_products=num_products)

    # Serialize blockchain and products
    blockchain_data = serialize_blockchain(scm.blockchain.to_dict())
    products_data = serialize_products(scm.products)

    # Prepare data to store
    data = {
        'blockchain': blockchain_data,
        'products': products_data
    }

    # Update simulation status
    status = f"Simulation run {n_clicks}: {num_products} products processed with {success_rate}% success rate."

    return data, status

# Callback to render visualizations based on selected tab and simulation data
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value'),
    State('simulation-data', 'data')
)
def render_content(tab, data):
    if not data:
        return html.Div("No simulation data available. Please run the simulation.")

    blockchain_data = data.get('blockchain', [])
    products_data = data.get('products', {})

    if tab == 'tab-1':
        fig = visualize_product_flow_data(products_data)
        return dcc.Graph(figure=fig)
    elif tab == 'tab-2':
        fig = visualize_blockchain_data(blockchain_data)
        return dcc.Graph(figure=fig)
    elif tab == 'tab-3':
        fig = visualize_entity_network_data(products_data)
        return dcc.Graph(figure=fig)
    elif tab == 'tab-4':
        fig = generate_gantt_chart_data(products_data)
        return dcc.Graph(figure=fig)
    else:
        return html.Div("Invalid Tab")

if __name__ == '__main__':
    app.run_server(debug=True)